In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = list()

for i in snakemake.input['runtime']:
    _df = pd.read_csv(i)
    i = i.split('_')
    _df['model'] = i[2].replace('model=', '')
    _df['Quantization'] =  i[3].replace('q=', '')
    df.append(_df)
    
df = pd.concat(df)

In [ ]:
df['Efficient'] = df['model'].str.endswith('e')
df['Model Size'] = df['model'].str.replace('e', '')

In [ ]:
plt.figure(figsize=(6, 2), dpi=300)

ax = sns.lineplot(data=df[(df['Quantization'] == 'False') & (df['mem_gb'] != -1)], 
                  x='length', y='mem_gb', hue='Model Size', style='Efficient')
sns.despine()
plt.xlabel('Protein Length')
plt.ylabel('Memory Usage (GB)')

oom_limit = 49.1
plt.axhline(y=oom_limit, linestyle='--', color='black')
plt.ylim(-1, oom_limit + 5)

import random
_df = df[df['mem_gb'] == -1] 
for i, row in enumerate(_df.itertuples()):
    print(row)
    ax.text(row.length, oom_limit + 1, 'OOM', 
            color=palette[row.model], ha='center', rotation=60)

plt.legend([],[], frameon=False)
plt.savefig(snakemake.output['fig'], bbox_inches='tight', dpi=300)

In [ ]:
plt.figure(figsize=(6, 2), dpi=300)
sns.boxplot(data=df[df['Efficient']], x='Model Size', y='runtime', hue='Quantization')
sns.despine()
plt.ylabel('Runtime (Seconds)')
plt.savefig(snakemake.output['fig_quantize'], bbox_inches='tight', dpi=300)